### Loading Dependencies

In [1]:
#using Pkg
#Pkg.add("LinearAlgebra")
#Pkg.add("Distributions")
#Pkg.add("Random")
#Pkg.add("NLsolve")
using LinearAlgebra,NLsolve, Random, Distributions

# Pure Strategy Equilibria

We start by defining basic functions. These functions are given by:
* sig_mi(i,K): This function returns $K_{-i}$ for a given $K$.
* sig_prof(i,k,K): This function returns the vector $(K_1,K_2,\dots,K_{i-1},k,K_{i+1},\dots,K_{N})$ for a given $i,k,K$
* Utmp(i,K,Room_pay): Returns a vector where component $l$ represents prize the player $i$ will recieve if they choose $l$ for a given opponent action profile stored in K. Room_pay can be varied here. 

In [2]:
#Function to return vector of opponent strategy profile
function sig_mi(i,K)
    Kret=ones(length(K)-1)
    kind=1
    for j=1:length(K)
        if(j!=i)
            Kret[kind]=K[j]
            kind=kind+1
        end
    end
    return Kret
end
#Function to return vector of opponent strategy profile
function sig_prof(i,k,K)
    Kret=ones(length(K))
    for j=1:length(K)
        if(j!=i)
            Kret[j]=K[j]
        end
        if(j==i)
            Kret[j]=k
        end
    end
    return Kret
end
#Function to return player utility vector
function Utmp(i,K,Room_pay)
    U_vec=ones(length(Room_pay))
    K_mi=sig_mi(i,K)
    for k=1:length(Room_pay)
        Ktmp=vcat(k,K_mi)
        U_vec[k]=Room_pay[k]./sum((Ktmp.==k))
    end
    return U_vec
end

Utmp (generic function with 1 method)

##  Algorithm
The algorithm is as followed:
* Guess $K_o$
* Let $U_{i,o}\leftarrow U_{i}(K_o)$ for all $i=1,2,\dots,N$
* Let $U_{i,n}\leftarrow 1$ for all $i=1,2,\dots,N$
* $t_{check}=0$
* while $\left\lVert U_{n}-U_{o}\right\rVert>0$ do
    - for i=1:N
        + if($t_{check}>0$)
            * $U_o\leftarrow U_{n}$
            * $K_o\leftarrow K_{n}$
        + end
        + $U_{i,n} \leftarrow \max_{k} \left\{U_i(k,K_{-i,o})\right\}$
        + $K_{i,n} \leftarrow \arg\max_{k} \left\{U_i(k,K_{-i,o})\right\}$
        + $t_{check}\leftarrow t_{check}+1$
    + end
* end

The algorithm Terminates in finie steps if a PSNE exists     

In [3]:
#The algorithm
function Find_PSNE(Num_p,K_old,Room_pay)
    #Initial Pay_off
    U_old=ones(Num_p)
    for i=1:length(K_old)
        U_old[i]=Room_pay[K_old[i]]./sum((K_old.==K_old[i]))
    end
    iters=0
    checks=0
    U_nxt=ones(Num_p)
    K_nxt=ones(Num_p)
    while  norm(U_nxt-U_old)>0
        for i=1:Num_p
            if(checks>0)
                U_old[:]=U_nxt[:]
                K_old[:]=K_nxt[:]
            end
            U_vec=Utmp(i,K_old,Room_pay)
            m_Utmp,m_ktmp=findmax(U_vec)
            K_nxt=sig_prof(i,m_ktmp,K_old)
            U_nxt=sig_prof(i,m_Utmp,U_old)
            checks+=1
        end
        iters+=1
    end
    return U_nxt,K_nxt
end

Find_PSNE (generic function with 1 method)

# RUNNING THE CODE
### Specify initial requirements

In [4]:
#Specify Number of Players
Num_p=8
#Specify Room specific pay-off function
Room_pay=vcat(40,50*ones(3),70*ones(3),100);
#Strating Guess
K_old=8 .*ones(Int64,Num_p)
print("The Guess Profile is Given By\n")
for i=1:Num_p
    print(string("Player ",i," initial Room Number ",K_old[i],"\n"))
end

The Guess Profile is Given By
Player 1 initial Room Number 8
Player 2 initial Room Number 8
Player 3 initial Room Number 8
Player 4 initial Room Number 8
Player 5 initial Room Number 8
Player 6 initial Room Number 8
Player 7 initial Room Number 8
Player 8 initial Room Number 8


### Run Code to find one equilibria

In [5]:
U_nxt,K_nxt=Find_PSNE(Num_p,K_old,Room_pay);
print("The Equilibrium Profile is Given By\n")
for i=1:Num_p
    print(string("Player ",i," Chooses Room Number ",K_nxt[i],". Their prize is ",U_nxt[i],"\n"))
end

The Equilibrium Profile is Given By
Player 1 Chooses Room Number 5.0. Their prize is 70.0
Player 2 Chooses Room Number 6.0. Their prize is 70.0
Player 3 Chooses Room Number 7.0. Their prize is 70.0
Player 4 Chooses Room Number 2.0. Their prize is 50.0
Player 5 Chooses Room Number 3.0. Their prize is 50.0
Player 6 Chooses Room Number 4.0. Their prize is 50.0
Player 7 Chooses Room Number 8.0. Their prize is 50.0
Player 8 Chooses Room Number 8.0. Their prize is 50.0


### Another Equilibria

In [6]:
#Specify Number of Players
Num_p=8
#Specify Room specific pay-off function
Room_pay=vcat(40,50*ones(3),70*ones(3),100);
#Starting Guess
K_old=1 .*ones(Int64,Num_p)
print("The Guess Profile is Given By\n")
for i=1:Num_p
    print(string("Player ",i," initial Room Number ",K_old[i],"\n"))
end
U_nxt,K_nxt=Find_PSNE(Num_p,K_old,Room_pay);
print("The Equilibrium Profile is Given By\n")
for i=1:Num_p
    print(string("Player ",i," Chooses Room Number ",K_nxt[i],". Their prize is ",U_nxt[i],"\n"))
end

The Guess Profile is Given By
Player 1 initial Room Number 1
Player 2 initial Room Number 1
Player 3 initial Room Number 1
Player 4 initial Room Number 1
Player 5 initial Room Number 1
Player 6 initial Room Number 1
Player 7 initial Room Number 1
Player 8 initial Room Number 1
The Equilibrium Profile is Given By
Player 1 Chooses Room Number 8.0. Their prize is 50.0
Player 2 Chooses Room Number 5.0. Their prize is 70.0
Player 3 Chooses Room Number 6.0. Their prize is 70.0
Player 4 Chooses Room Number 7.0. Their prize is 70.0
Player 5 Chooses Room Number 2.0. Their prize is 50.0
Player 6 Chooses Room Number 3.0. Their prize is 50.0
Player 7 Chooses Room Number 4.0. Their prize is 50.0
Player 8 Chooses Room Number 8.0. Their prize is 50.0


# Mixed Strategy Equilibrium

### Search for Symmetric Mixed Strategy Equilibria

Recall the indifference conditions we use that and the fact $\sigma_1,\sigma_2,\dots,\sigma_8$ is a probability mass function.

Suppose player $1$ choose room $k$, given his opponents choose room $k$ with probability $\sigma_k$ each independently. Then his expected utility is given by:
$$U_{1}(k;\sigma_{-i})=\sum_{x=0}^7\frac{m_k}{x+1}\binom{N-1}{x}\sigma_k^x(1-\sigma_k)^{N-1-X} $$

We solve for equations:
$$U_{i}-U_{1}=0 \hspace{4mm} \forall \hspace{2mm}i=2,3,\dots,8 $$
and 
$$1-\sum_{i=1}^8\sigma_i=0$$

We have 8 equations and 8 unknowns

In [7]:
function ret_res(Num_P,Room_pay,sig_strat)
    Ret_vec=zeros(8)
    Pay_offRm=zeros(8)
    for l=1:8
        for x=0:7
            Pay_offRm[l]+=Room_pay[l]/(x+1)*binomial(Num_p-1,x)*sig_strat[l]^x*(1-sig_strat[l])^(Num_p-1-x)
        end
    end
    for l=2:8
        Ret_vec[l-1]=Pay_offRm[l]-Pay_offRm[1]
    end
    Ret_vec[8]=sum(sig_strat)-1;
    return Ret_vec
end

ret_res (generic function with 1 method)

In [8]:
function solve_fun(sig_strat)
    return ret_res(Num_p,Room_pay,sig_strat)
end
initial_guess= ones(8)./8
soln_model=nlsolve(solve_fun,initial_guess,show_trace=true)
soln_model.zero

Iter     f(x) inf-norm    Step 2-norm 
------   --------------   --------------
     0     3.938347e+01              NaN
     1     3.535852e+00     2.295869e-01
     2     2.354455e-01     4.459443e-02
     3     1.082001e-03     2.433612e-03
     4     1.459777e-08     9.001552e-06
     5     1.421085e-14     1.263819e-10


8-element Vector{Float64}:
 0.0005900395709165937
 0.06553637288318381
 0.06553637288318374
 0.06553637288318374
 0.16967796808971655
 0.16967796808971655
 0.16967796808971655
 0.2937669375103824

# Mixed Strategy with theoretical assistance

The payoff in room 1 is 40.  In any pure strategy equilibrium. a player who os evaluating knows where every other player is going to go.  Given the progile of strategies the others are using, and one of the other rooms is empty, then the player will prefer the empty room because all other rooms have payoffs above 40. The only other possibility is that each other room is being chosen by at least one player.  Then 7 rooms and 7 players mean that in this case each room has exactly one player, which makes the payoff to visiting room 8 50 > 40. So room 1 is strictly dominated.

This simplifies the symmetric mixed strategy equilibrium because we can set $\pi_1 = 0$.

The second simplification comes from the fact that rooms 2,3,4 are identical, and rooms 5,6,7 are identical.  Then we can assume 
$$
\pi_2 = \pi_3 = \pi_4
$$
and 
$$
\pi_5=\pi_6=\pi_7
$$
To find the symmetric Nash equilibrium, solve
$$
\sum_{i=0}^{7}\frac{7!}{i!(7-i)!}\pi_2^i (1-\pi_2)^{7-i} \frac{50}{i+1} = \sum_{i=0}^{7}\frac{7!}{i!(7-i)!}\pi_5^i (1-\pi_5)^{7-i} \frac{70}{i+1}
$$

$$
\sum_{i=0}^{7}\frac{7!}{i!(7-i)!}\pi_2^i (1-\pi_2)^{7-i} \frac{50}{i+1} = \sum_{i=0}^{7}\frac{7!}{i!(7-i)!}\pi_8^i (1-\pi_8)^{7-i} \frac{100}{i+1}
$$
along with the equation
$$
\pi_8 = 1 -3\pi_2 - 3\pi_5
$$

In [7]:
#using Pkg
#Pkg.add("SymPy")
using SymPy
pi_2,pi_5,pi_8 = symbols("pi_2,pi_5, pi_8")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`


(pi_2, pi_5, pi_8)

In [16]:
function bisum(x,r)
    sum = 0
    for i in 0:7
        sum += (factorial(7)/(factorial(i)*factorial(7-i))) *x^i * (1-x)^(7-i) * r/(i+1)
    end
    return sum
end
        

bisum (generic function with 1 method)

In [20]:
bisum(pi_2, 50)

       7          6                    5         2           4         3      
6.25⋅π₂  + 50.0⋅π₂ ⋅(1 - π₂) + 175.0⋅π₂ ⋅(1 - π₂)  + 350.0⋅π₂ ⋅(1 - π₂)  + 437

     3         4           2         5                    6                7
.5⋅π₂ ⋅(1 - π₂)  + 350.0⋅π₂ ⋅(1 - π₂)  + 175.0⋅π₂⋅(1 - π₂)  + 50.0⋅(1 - π₂) 

If you use solve on this system it won't find solutions, so you need nsolve

In [27]:
nsolve([bisum(pi_2,50)-bisum(pi_5,70),bisum(pi_2,50)-bisum(pi_8,100),pi_8-(1-3*pi_2-3*pi_5)], (pi_2,pi_5,pi_8),(1/3,1/3,1/3))

3×1 Matrix{Sym}:
 0.06561457713648978591554449395254166359707576078440925818030393800964737792640
  0.1697634105296091051109250028257818667174966948010267516043503133590974799449
  0.2938660370017033269205915096650294090562826332436919706460372458937654263863